============================================================================================

# Extra Tree


============================================================================================


In [1]:
import warnings

warnings.filterwarnings('ignore', category=DeprecationWarning)

In [ ]:
import pandas as pd
import numpy as np
import os
import re
import random
from matplotlib import pyplot as plt
from sklearn.ensemble import ExtraTreesClassifier


from sklearn.model_selection import cross_val_score
from bayes_opt import BayesianOptimization

from sklearn.decomposition import PCA

import fns_models as fns


% matplotlib inline

from subprocess import check_output
print(check_output(["ls", "data"]).decode("utf-8"))

In [ ]:
train, train_labels, test, test_labels = fns.get_top_author(3)


================================================================================================================

# Bayesian Optimization + Extra Tree

[bayesian-optimization](https://github.com/fmfn/BayesianOptimization/blob/master/bayes_opt/bayesian_optimization.py)

================================================================================================================

In [ ]:
def extr_evaluate(max_features, max_depth, n_estimators):
    
    random.seed(2017)
#     params['max_features'] = int(max_features)
#     params['max_depth'] = int(max_depth)
#     params['n_estimators'] = int(n_estimators)
    
    extrc = ExtraTreesClassifier(n_jobs = 4, n_estimators=int(n_estimators), oob_score=True,
                                max_depth = int(max_depth), max_features = int(max_features),
                                random_state = 2017)
    #scores = cross_val_score(extrc, X=train, y = train_labels, cv=5, n_jobs = 2)
    
    # The mean score and the 95% confidence interval of the score estimate are hence given by:
    #print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))
    #return scores.mean()
    extrc.fit(train, train_labels)
    return extrc.oob_score_

def extr_pca_evaluate(max_features,
                    max_depth,
                    n_estimators):
    
    
    extrc = ExtraTreesClassifier(n_jobs = 4, n_estimators=int(n_estimators), oob_score=True,
                                max_depth = int(max_depth), max_features = int(max_features),
                                random_state = 2017)
    
    extrc.fit(pca_transformed, train_labels)
    return extrc.oob_score_

def extr_bo(extr_fnc=extr_evaluate):
    num_iter = 25
    init_points = 5
    random_state = 2017
    # params = {
    #     #'eta': 0.1,
    #     #'silent': 1,
    #     'eval_metric': 'mae',
    #     'verbose_eval': True,
    #     #'seed': random_state
    # }

    extrBO = BayesianOptimization(extr_fnc, {'max_features': (5, 7),
                                             'max_depth': (2, 10),
                                             'n_estimators': (100, 900)})

    extrBO.maximize(init_points=init_points, n_iter=num_iter)
    
    print "Bayesian Optimization Best Score: %d" % extrBO.res['max']['max_val']

    print "Bayesian Optimization Best Parameters: %s" % str(extrBO.res['max']['max_params'])
    
    print (extrBO.res['max'])
    
    fns.plot_bo(extr_fnc, extrBO)

#     print "Bayesian Optimization  Parameters: %s" % str(extrBO.res['all'])



In [ ]:
# Run BO for color histogram
extr_bo(extr_evaluate)

In [ ]:
# Choose the best params from BO results then predict 
best_estimators = 
best_max_depth = 
best_max_features = 
best_extrc = ExtraTreesClassifier(n_jobs = 4, n_estimators=best_estimators, oob_score=False,
                                max_depth = best_max_depth, max_features = best_max_features)

best_extrc.fit(train, train_labels)

# use the best params to predict
extrc_true, extrc_pred = test_labels, best_extrc.predict(test)
extrc_pred

In [ ]:
test_data_df = fns.result_table(extrc_true, extrc_pred)
test_data_df

================================================================================================================

# PCA + Extra Tree

================================================================================================================

In [ ]:
## Get 15 principal components
pca = PCA(n_components=15)
pca.fit(train)
pca_transformed = pca.transform(train)

pca_transformed_test = pca.transform(test)

In [ ]:
plt.plot(pca.explained_variance_ratio_)
plt.title("Scree Plot: 10 Principal Components")
plt.xlabel("Number of Components")
plt.ylabel("% of Explained Variance")
sum(pca.explained_variance_ratio_)

In [ ]:
# Run BO for pca of color histogram
extr_bo(extr_pca_evaluate)

In [ ]:
# Choose the best params from BO results then predict 
best_estimators = 
best_max_depth = 
best_max_features = 
best_pca_extrc = ExtraTreesClassifier(n_jobs = 4, n_estimators=best_estimators, oob_score=False,
                                max_depth = best_max_depth, max_features = best_max_features)

best_pca_extrc.fit(train, train_labels)

# use the best params to predict
extrc_pca_true, extrc_pca_pred = test_labels, best_extrc.predict(test)
extrc_pca_pred

In [ ]:
# accuracy of training dataset
best_pca_extrc.score(pca_transformed, train_labels)

In [ ]:
# accuracy of testing dataset
best_pca_extrc.score(pca_transformed_test, test_labels)

In [ ]:
test_data_df = result_table(extrc_pca_true, extrc_pca_pred)
test_data_df